# External Data Nagoya

EO: https://apps.sentinel-hub.com/eo-browser/?zoom=11&lat=34.78867&lng=136.55055&themeId=DEFAULT-THEME

In [1]:
import os
import warnings
import random
from pprint import pprint
import copy
from typing import List, Tuple
import glob
import json
import csv
# import dataclasses
from joblib import Parallel, delayed
from typing import List, Set, Dict, Any

from tqdm import tqdm
import numpy as np
import pandas as pd
from box import Box, from_file
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from scipy.optimize import minimize
from sklearn.metrics import jaccard_score
import cv2
import seaborn as sns

In [2]:
class CFG(object):
    # basic
    debug: bool = False
    folds: int  = 4
    seed: int   = 417
    eps: float  = 1e-12
    
    # data
    PATH_CSV: str  =  f'../data/021_Nagoya/019_seed{seed}.csv'
    PATH_NPY_ROOT = f'../data/021_Nagoya/'
    PATH_DATA = '../data/Nagoya/'
    os.makedirs(PATH_DATA, exist_ok=True), os.makedirs(PATH_NPY_ROOT + 'img/', exist_ok=True)
    
# box
cfg = Box({k:v for k, v in dict(vars(CFG)).items() if '__' not in k})
    
# 乱数のシードを設定
np.random.seed(cfg.seed)
random.seed(cfg.seed)
    
pprint(cfg)

{'PATH_CSV': '../data/021_Nagoya/019_seed417.csv',
 'PATH_DATA': '../data/Nagoya/',
 'PATH_NPY_ROOT': '../data/021_Nagoya/',
 'debug': False,
 'eps': 1e-12,
 'folds': 4,
 'seed': 417}


## CLI

In [3]:
!aws --version

aws-cli/1.27.131 Python/3.8.16 Linux/5.15.0-69-generic botocore/1.29.131


## Download

`2020/04/30` - `2023/05/30` 

few clous scene

In [4]:
PATHS_SENTILEL2_S3 = [
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/20/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/10/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2023/3/16/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2023/3/11/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2023/3/6/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2023/2/4/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2023/1/20/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2022/11/6/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2022/10/2/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2022/8/3/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2022/6/29/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2022/6/4/0/",
    "s3://sentinel-s2-l2a/tiles/53/S/PU/2022/4/20/0/",
]

In [8]:
for PATH_S3 in tqdm(PATHS_SENTILEL2_S3):
    SCENE_ID = PATH_S3.split('tiles')[1].replace('/', '-')[1:-3]
    print(SCENE_ID)
    OUTPUT_PATH = f'{cfg.PATH_DATA}/{SCENE_ID}/'
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    
    !aws s3 cp {PATH_S3} {OUTPUT_PATH} --recursive --request-payer
    # break

  0%|          | 0/13 [00:00<?, ?it/s]

53-S-PU-2023-4-20
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/20/0/R10m/AOT.jp2 to ../data/Nagoya/53-S-PU-2023-4-20/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/20/0/R10m/B04.jp2 to ../data/Nagoya/53-S-PU-2023-4-20/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/20/0/R10m/B03.jp2 to ../data/Nagoya/53-S-PU-2023-4-20/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/20/0/R20m/AOT.jp2 to ../data/Nagoya/53-S-PU-2023-4-20/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/20/0/R10m/B02.jp2 to ../data/Nagoya/53-S-PU-2023-4-20/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/20/0/R10m/B08.jp2 to ../data/Nagoya/53-S-PU-2023-4-20/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/20/0/R10m/TCI.jp2 to ../data/Nagoya/53-S-PU-2023-4-20/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/20/0/R20m/B01.jp2 to ../data/Nagoya/53-S-PU-2023-4-20/R20m/B01.jp2
download: s3://sentinel-s2-l2a

  8%|▊         | 1/13 [04:08<49:44, 248.73s/it]

53-S-PU-2023-4-10
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/10/0/R10m/AOT.jp2 to ../data/Nagoya/53-S-PU-2023-4-10/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/10/0/R10m/B03.jp2 to ../data/Nagoya/53-S-PU-2023-4-10/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/10/0/R10m/B04.jp2 to ../data/Nagoya/53-S-PU-2023-4-10/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/10/0/R20m/AOT.jp2 to ../data/Nagoya/53-S-PU-2023-4-10/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/10/0/R10m/B02.jp2 to ../data/Nagoya/53-S-PU-2023-4-10/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/10/0/R20m/B01.jp2 to ../data/Nagoya/53-S-PU-2023-4-10/R20m/B01.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/10/0/R10m/B08.jp2 to ../data/Nagoya/53-S-PU-2023-4-10/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2023/4/10/0/R10m/TCI.jp2 to ../data/Nagoya/53-S-PU-2023-4-10/R10m/TCI.jp2
download: s3://sentinel-s2-l2a

 15%|█▌        | 2/13 [10:09<57:39, 314.48s/it]

...

 85%|████████▍ | 11/13 [1:12:54<13:12, 396.35s/it]

53-S-PU-2022-6-4
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/6/4/0/R10m/AOT.jp2 to ../data/Nagoya/53-S-PU-2022-6-4/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/6/4/0/R10m/B02.jp2 to ../data/Nagoya/53-S-PU-2022-6-4/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/6/4/0/R20m/AOT.jp2 to ../data/Nagoya/53-S-PU-2022-6-4/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/6/4/0/R10m/B04.jp2 to ../data/Nagoya/53-S-PU-2022-6-4/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/6/4/0/R10m/B03.jp2 to ../data/Nagoya/53-S-PU-2022-6-4/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/6/4/0/R10m/TCI.jp2 to ../data/Nagoya/53-S-PU-2022-6-4/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/6/4/0/R10m/B08.jp2 to ../data/Nagoya/53-S-PU-2022-6-4/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/6/4/0/R20m/B01.jp2 to ../data/Nagoya/53-S-PU-2022-6-4/R20m/B01.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/20

 92%|█████████▏| 12/13 [1:17:41<06:03, 363.14s/it]

53-S-PU-2022-4-20
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/4/20/0/R10m/AOT.jp2 to ../data/Nagoya/53-S-PU-2022-4-20/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/4/20/0/R10m/B04.jp2 to ../data/Nagoya/53-S-PU-2022-4-20/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/4/20/0/R10m/B03.jp2 to ../data/Nagoya/53-S-PU-2022-4-20/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/4/20/0/R10m/B02.jp2 to ../data/Nagoya/53-S-PU-2022-4-20/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/4/20/0/R20m/AOT.jp2 to ../data/Nagoya/53-S-PU-2022-4-20/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/4/20/0/R10m/TCI.jp2 to ../data/Nagoya/53-S-PU-2022-4-20/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/4/20/0/R10m/B08.jp2 to ../data/Nagoya/53-S-PU-2022-4-20/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/53/S/PU/2022/4/20/0/R20m/B01.jp2 to ../data/Nagoya/53-S-PU-2022-4-20/R20m/B01.jp2
download: s3://sentinel-s2-l2a

100%|██████████| 13/13 [1:22:00<00:00, 378.47s/it]


## Loading data

In [9]:
BANDS = [
    'R20m/B01', 'R10m/B02', 'R10m/B03', 'R10m/B04', 'R20m/B05', 'R20m/B06', 
    'R20m/B07', 'R10m/B08', 'R20m/B8A', 'R60m/B09', 'R20m/B11', 'R20m/B12',
]

In [10]:
PATHS_NPY = []

for PATH_S3 in tqdm(PATHS_SENTILEL2_S3):
    SCENE_ID = PATH_S3.split('tiles')[1].replace('/', '-')[1:-3]
    print(SCENE_ID)
    OUTPUT_PATH = f'{cfg.PATH_DATA}/{SCENE_ID}/'
    
    imgs = []
    
    for BAND in BANDS:
        PATH_BAND = f'{OUTPUT_PATH}{BAND}.jp2'
        img = cv2.imread(PATH_BAND, cv2.IMREAD_UNCHANGED)        
        if 'R20m' in BAND:
            img = cv2.resize(img, (img.shape[1]*2, img.shape[0]*2), interpolation=1)
        elif 'R60m' in BAND:
            img = cv2.resize(img, (img.shape[1]*6, img.shape[0]*6), interpolation=1)

        imgs.append(img)
        
    imgs = np.stack(imgs, axis=2)
    
    for h_idx in range(imgs.shape[0]//512):
        for w_idx in range(imgs.shape[1]//512):
            
            patch = imgs[h_idx*512:h_idx*512+512, w_idx*512:w_idx*512+512, :]
                
            PATH_NPY = f'{cfg.PATH_NPY_ROOT}img/{SCENE_ID}_h{h_idx}_w{w_idx}.npy'
            np.save(PATH_NPY, patch)
            PATHS_NPY.append(PATH_NPY)


  0%|          | 0/13 [00:00<?, ?it/s]

53-S-PU-2023-4-20


  8%|▊         | 1/13 [02:39<31:57, 159.80s/it]

53-S-PU-2023-4-10


 15%|█▌        | 2/13 [05:11<28:26, 155.10s/it]

...

 85%|████████▍ | 11/13 [29:50<05:14, 157.09s/it]

53-S-PU-2022-6-4


 92%|█████████▏| 12/13 [32:17<02:34, 154.04s/it]

53-S-PU-2022-4-20


100%|██████████| 13/13 [34:39<00:00, 159.98s/it]


In [11]:
df = pd.DataFrame({
    'feature': PATHS_NPY
})

df['name'] = df['feature'].apply(lambda x: x.split('/')[-1].split('.')[0])
df['mask'] = False
df['defects'] = False
df['fold'] = -1
df['hight'] = 512
df['width'] = 512
df['california'] = False
df['cv'] = -1
df['area'] = 'nagoya'

df.head(4)

,feature,name,mask,defects,fold,hight,width,california,cv,area
0,../data/021_Nagoya/img/53-S-PU-2023-4-20_h0_w0...,53-S-PU-2023-4-20_h0_w0,False,False,-1,512,512,False,-1,nagoya
1,../data/021_Nagoya/img/53-S-PU-2023-4-20_h0_w1...,53-S-PU-2023-4-20_h0_w1,False,False,-1,512,512,False,-1,nagoya
2,../data/021_Nagoya/img/53-S-PU-2023-4-20_h0_w2...,53-S-PU-2023-4-20_h0_w2,False,False,-1,512,512,False,-1,nagoya
3,../data/021_Nagoya/img/53-S-PU-2023-4-20_h0_w3...,53-S-PU-2023-4-20_h0_w3,False,False,-1,512,512,False,-1,nagoya


In [12]:
df.to_csv(cfg.PATH_CSV, index=False, header=True)